In [61]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#importing libraries

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression , Ridge, Lasso
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler


/kaggle/input/home-data-for-ml-course/sample_submission.csv
/kaggle/input/home-data-for-ml-course/sample_submission.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv.gz
/kaggle/input/home-data-for-ml-course/data_description.txt
/kaggle/input/home-data-for-ml-course/test.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv
/kaggle/input/home-data-for-ml-course/test.csv


THis is my first model, I had to learnt so many things and there are so many things to learn. I took help from AI, But I will get back stronger :)

In [62]:
#loading data from csv to df

train = pd.read_csv('/kaggle/input/home-data-for-ml-course/train.csv')

X = train.drop('SalePrice', axis=1)
y = train['SalePrice']

In [63]:
#splitting the columns, There are columns which stores int and float values and there are columns which stores "object"values like Neighbourhood

num_cols = X.select_dtypes(include=["int64","float64"]).columns

cat_cols = X.select_dtypes(include=["object"]).columns

In [64]:
#filling tha NAN values with median and then scaling it

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

#filling the Nans with most frequent, then one-hot encoding it

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

#Column Transformer: apply different pipleline to num/cat

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer,cat_cols)
    ]
)


#Final Model Pipeline

linreg_model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", LinearRegression())
])

ridge_model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", Ridge(alpha=1.0,solver="auto"))
])

lasso_model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", Lasso(alpha=0.001))
])

In [65]:
#train test and split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [66]:
for name, model in [("Linear", linreg_model),("Ridge", ridge_model),("Lasso", lasso_model)]:
    model.fit(X_train, y_train)
    y_pred = model.predict(X)

    rmse = np.sqrt(mean_squared_error(y_train, y_pred))
    mae = mean_absolute_error(y_train, y_pred)
    r2 = r2_score(y_train, y_pred)


ValueError: Found input variables with inconsistent numbers of samples: [1168, 1460]

In [ ]:
print(f"\n{name} Regression:")
print("RMSE:", rmse)
print("MAE :", mae)
print("R2  :",r2)

In [ ]:
model.fit(X,y)

test_data = pd.read_csv('/kaggle/input/home-data-for-ml-course/test.csv')

test_preds = model.predict(test_data)

submission = test_data[['Id']]
submission


In [ ]:
submission["SalePrice"] = test_preds

In [68]:
submission.to_csv("submission.csv", index=None)

In [69]:
pd.read_csv("submission.csv")

,Id,SalePrice
0,1461,113640.685747
1,1462,155074.204771
2,1463,187958.864507
3,1464,201773.492908
4,1465,202921.039491
...,...,...
1454,2915,81762.206402
1455,2916,81446.597516
1456,2917,181861.043815
1457,2918,109881.397335
